In [1]:
import os
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Path to your ChromeDriver
driver_path = "/usr/local/bin/chromedriver"

# Set up the WebDriver with Chrome
driver = webdriver.Chrome(service=Service(driver_path))

# Folder containing the images
image_folder = "/Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw"

# Base folder where processed images will be saved
output_base_folder = "/Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed"

# List all image files in the folder (you can filter by extension if needed)
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpeg', '.jpg', '.png'))]

# Polling time interval (in seconds)
polling_interval = 10

# Maximum polling time (in seconds), set this to a large value or remove it if you want infinite polling
max_polling_time = 1800  # 30 minutes

# Loop through each image
for image_file in image_files:
    try:
        # Construct the full path to the image
        file_path = os.path.join(image_folder, image_file)

        # Check if the file exists
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            continue

        print(f"Processing image: {file_path}")

        # Navigate to the Retina Vessel website for each image
        driver.get("http://www.retinavessel.com")

        # Wait until the file input element is available
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input.el-upload__input"))
        )

        # Find the file upload input element and upload the image
        file_input = driver.find_element(By.CSS_SELECTOR, "input.el-upload__input")

        # Send the file path to the input field
        file_input.send_keys(file_path)

        # Polling for the images to appear
        start_time = time.time()
        images_generated = False  # Flag to check if images are generated

        while True:
            try:
                # Try to find the images and labels generated on the page
                image_elements = driver.find_elements(By.CSS_SELECTOR, "img.el-image__inner")
                labels = driver.find_elements(By.CSS_SELECTOR, "span.demonstration")

                # If 6 images and labels are found, break the loop
                if len(image_elements) >= 6 and len(labels) >= 6:
                    print(f"Images have been generated for {image_file}")
                    images_generated = True
                    break

                # Check if max polling time has been exceeded
                elapsed_time = time.time() - start_time
                if elapsed_time > max_polling_time:
                    print(f"Maximum polling time exceeded for {image_file}")
                    break

                # Wait before polling again
                time.sleep(polling_interval)

            except NoSuchElementException:
                print(f"Error locating images or labels on the page for {image_file}. Retrying...")
                time.sleep(polling_interval)
                continue

        # If no images were generated, log the failure and continue
        if not images_generated:
            print(f"Failed to generate images for {image_file}. Skipping to the next file.")
            continue

        # Create a folder for the current image file
        image_output_folder = os.path.join(output_base_folder, os.path.splitext(image_file)[0])
        os.makedirs(image_output_folder, exist_ok=True)

        # Download each generated image with its respective label
        print(f"Generated image URLs for {image_file}:")
        for idx, img in enumerate(image_elements):
            image_url = img.get_attribute("src")
            label = labels[idx].text  # Get the corresponding label for each image
            print(f"Image {idx + 1} ({label}): {image_url}")

            # Fetch the image data
            try:
                response = requests.get(image_url)

                # Create a filename using the label (e.g., "A/V_ratio.png")
                sanitized_label = label.replace(' ', '_').replace('/', '_')  # Sanitize the label for filenames
                output_image_path = os.path.join(image_output_folder, f"{sanitized_label}.png")

                # Save the image to the folder
                with open(output_image_path, 'wb') as f:
                    f.write(response.content)

                print(f"Downloaded image {idx + 1} ({label}) to {output_image_path}")

            except Exception as download_error:
                print(f"Error downloading image {idx + 1} for {image_file}: {download_error}")

        print(f"Completed processing of {image_file}\n")

    except TimeoutException as e:
        print(f"Timeout while processing {image_file}: {e}")
    except Exception as general_error:
        print(f"An error occurred while processing {image_file}: {general_error}")

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10594_left.jpeg
Images have been generated for 10594_left.jpeg
Generated image URLs for 10594_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726445551961.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10594_left/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726445551961_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10594_left/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726445551961_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10594_left/AV_crossing.png
Image 4 (A/V ratio): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output

Images have been generated for 10475_left.jpeg
Generated image URLs for 10475_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726445773596.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10475_left/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726445773596_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10475_left/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726445773596_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10475_left/AV_crossing.png
Image 4 (A/V ratio): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726445773596_avr.png
Downloaded image 4 (A/V ratio) to /Users/chunduri/Desktop/CVD Project/Dat

Images have been generated for 10426_right.jpeg
Generated image URLs for 10426_right.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726446578327.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10426_right/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446578327_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10426_right/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446578327_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10426_right/AV_crossing.png
Image 4 (A/V ratio): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446578327_avr.png
Downloaded image 4 (A/V ratio) to /Users/chunduri/Desktop/CVD Projec

Images have been generated for 10550_left.jpeg
Generated image URLs for 10550_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726446846432.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10550_left/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446846432_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10550_left/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446846432_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10550_left/AV_crossing.png
Image 4 (A/V ratio): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726446846432_avr.png
Downloaded image 4 (A/V ratio) to /Users/chunduri/Desktop/CVD Project/Dat

Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10603_right/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726447111019_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10603_right/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726447111019_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10603_right/AV_crossing.png
Image 4 (A/V ratio): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726447111019_avr.png
Downloaded image 4 (A/V ratio) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10603_right/A_V_ratio.png
Image 5 (Artery): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726447111019_A.png
Downloaded image 5 (Artery) to /U

...

Downloaded image 4 (A/V ratio) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10585_left/A_V_ratio.png
Image 5 (Artery): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726458231156_A.png
Downloaded image 5 (Artery) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10585_left/Artery.png
Image 6 (Vein): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726458231156_V.png
Downloaded image 6 (Vein) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10585_left/Vein.png
Completed processing of 10585_left.jpeg

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10646_left.jpeg
Images have been generated for 10646_left.jpeg
Generated image URLs for 10646_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726458293722.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Proces

Downloaded image 5 (Artery) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10705_right/Artery.png
Image 6 (Vein): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726458489924_V.png
Downloaded image 6 (Vein) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10705_right/Vein.png
Completed processing of 10705_right.jpeg

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10732_right.jpeg
Images have been generated for 10732_right.jpeg
Generated image URLs for 10732_right.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726458530957.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10732_right/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726458530957_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Data

Downloaded image 5 (Artery) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10541_left/Artery.png
Image 6 (Vein): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726459311181_V.png
Downloaded image 6 (Vein) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10541_left/Vein.png
Completed processing of 10541_left.jpeg

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10407_left.jpeg
Images have been generated for 10407_left.jpeg
Generated image URLs for 10407_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726459374457.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10407_left/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726459374457_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Ad

Downloaded image 5 (Artery) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10448_left/Artery.png
Image 6 (Vein): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726460147205_V.png
Downloaded image 6 (Vein) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10448_left/Vein.png
Completed processing of 10448_left.jpeg

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10403_right.jpeg
Images have been generated for 10403_right.jpeg
Generated image URLs for 10403_right.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726460205434.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10403_right/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726460205434_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Dataset

Downloaded image 6 (Vein) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10495_left/Vein.png
Completed processing of 10495_left.jpeg

Processing image: /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Raw/10389_left.jpeg
Images have been generated for 10389_left.jpeg
Generated image URLs for 10389_left.jpeg:
Image 1 (Input image): http://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/input/1726460456826.jpeg
Downloaded image 1 (Input image) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10389_left/Input_image.png
Image 2 (Segmentation): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726460456826_avoverl.png
Downloaded image 2 (Segmentation) to /Users/chunduri/Desktop/CVD Project/Datasets/Aditya/Week1_Processed/10389_left/Segmentation.png
Image 3 (AV crossing): https://retinavessel-file.oss-ap-southeast-2.aliyuncs.com/output/1726460456826_crossing.png
Downloaded image 3 (AV crossing) to /Users/chunduri/Deskto

In [2]:
driver.quit()